### Important:
This file must be run using **Anaconda** with the following packages:
- numpy
- pandas
- osgeo.gdal

In [1]:
from modules.environment_manager import get_notebook_path, get_persistent_config_data

NOTEBOOK_NAME = "geo_image_construction.ipynb"

NOTEBOOK_PATH = get_notebook_path(NOTEBOOK_NAME)
THREADS, MEMORY, HSI_CONFIG, PROJECT_FOLDER, OUTPUT_FOLDER = get_persistent_config_data(NOTEBOOK_PATH)

In [2]:
import math
import pandas as pd

In [3]:
df = pd.DataFrame()
for geo_fp in (PROJECT_FOLDER / "data" / "vp_geo").rglob("*"):
    if geo_fp.suffix == ".csv":
        geo_df = pd.read_csv(geo_fp)
        geo_df.columns = [col.strip().lower() for col in geo_df.columns]
        wavelengths, vp_name = geo_df.columns
        if not wavelengths == "wavelength":
            print("fail wavelength name check", geo_fp)
        if not vp_name.startswith("vp"):
            print("fail vp name check", geo_fp)
        if not math.isclose(geo_df["wavelength"][201], 899.41803):
            print("fail wavelength isclose check", geo_fp)
        if not geo_df[vp_name][:202].round(5).nunique() == 1:
            print("fail all same check", geo_fp)
        vp_num = int(vp_name[2:])
        new_vp_name = f"vp0{vp_num}" if vp_num < 10 else vp_name
        colname = geo_fp.parent.name
        df.loc[new_vp_name, colname] = geo_df[vp_name][0]
    elif geo_fp.is_dir():
        df[geo_fp.name] = None

assert "solar_azimuth" in df.columns
assert "sensor_azimuth" in df.columns

df["relative_azimuth"] = df["solar_azimuth"] - df["sensor_azimuth"]
df["relative_azimuth"] = (df["relative_azimuth"].mod(360) + 360).mod(360)  # adjust to be between 0 and 360
df = df.sort_index()
df = df.sort_index(axis=1)
df

,relative_azimuth,sensor_azimuth,sensor_zenith,solar_azimuth,solar_zenith
vp03,105.634543,4.895457,13.037619,110.53,35.53
vp04,105.644973,4.885027,10.211479,110.53,35.53
vp05,105.612238,4.907762,12.109447,110.52,35.53
vp06,105.689935,4.830065,9.422757,110.52,35.53
vp08,284.403807,186.116193,1.278386,110.52,35.53
vp09,285.495573,185.034427,10.772873,110.53,35.52
vp10,106.835811,3.674189,0.897489,110.51,35.53
vp11,102.853457,7.656543,4.640381,110.51,35.54
vp13,284.313169,185.646831,9.702342,109.96,35.95
vp14,284.200041,185.759959,6.265679,109.96,35.95


In [4]:
from osgeo import gdal